In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!unzip -q /content/font_fam-20220209T113134Z-001.zip
!unzip -q /content/font_fam-20220209T113134Z-002.zip

In [ ]:
from tqdm import tqdm
import os
name_file = []
for root, dirs, files in tqdm(os.walk("/content/font_fam")):
    for file in files:
        if file.endswith(".png"):
#              print(os.path.join(root, file))
             name_file.append(os.path.join(root, file))   

11it [00:00, 48.41it/s]


In [ ]:
nameItalic = []
nameNonItalic = []
italic = []
nonitalic = []
for i in name_file:
  if i.find('italic') > 0:
    italic.append(1)
    nameItalic.append(i)
  else:
    nonitalic.append(0)
    nameNonItalic.append(i)


In [ ]:
print(len(italic))
print(len(nonitalic))

43258
42824


In [ ]:
import pandas as pd
fontstyle = italic + nonitalic
nameImage = nameItalic + nameNonItalic
df = pd.DataFrame({'nameImage': nameImage, 'fontstyle': fontstyle})

In [ ]:
df

,nameImage,fontstyle
0,/content/font_fam/Kunlasatri/f57953fd_Kunlasat...,1
1,/content/font_fam/Kunlasatri/c892fda9_Kunlasat...,1
2,/content/font_fam/Kunlasatri/d5f3317e_Kunlasat...,1
3,/content/font_fam/Kunlasatri/ce543036_Kunlasat...,1
4,/content/font_fam/Kunlasatri/6e7bdf91_Kunlasat...,1
...,...,...
86077,/content/font_fam/TH_Chakra_Petch/c3f2bec4_TH_...,0
86078,/content/font_fam/TH_Chakra_Petch/86f8bf1_TH_C...,0
86079,/content/font_fam/TH_Chakra_Petch/409c55a4_TH_...,0
86080,/content/font_fam/TH_Chakra_Petch/98927ab4_TH_...,0


In [ ]:
dataFrame_pos = (df[df['fontstyle']==1]).sample(n=4000,random_state=101)
dataFrame_neg = (df[df['fontstyle']==0]).sample(n=4000,random_state=101)

new_df = pd.concat([dataFrame_pos,dataFrame_neg])

In [ ]:
import cv2
import numpy as np
from numpy import asarray
import matplotlib.pyplot as plt

def load_image(image_id):
    file_path = image_id
    image = cv2.imread(file_path)
    image = cv2.resize(image, (224, 224))



    return asarray(image)
# X = new_df['nameImage'][:].apply(load_image)
X = []
for i in tqdm(new_df['nameImage']):
  X.append(load_image(i))

100%|██████████| 8000/8000 [00:01<00:00, 5563.38it/s]


In [ ]:
# X = asarray(X.tolist())
X = asarray(X)
X = X.astype('float32')
X /= 255

In [ ]:
y = new_df['fontstyle']
import numpy as np
from sklearn.model_selection import train_test_split
# X = np.array(X)
y = np.array(y)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=99)


In [ ]:
# import cv2
# import matplotlib.pyplot as plt

# def load_image2(path):
    
#     image = cv2.imread(path)
#     # image = cv2.imread(')
#     # image = cv2.resize(image, (224, 224))
#     return image
    
# # def preprocess():


# # img_test = imageTest['id'][:].apply(load_image2)
# path = X_train[0]
# im = load_image2(path)
# plt.imshow(im)

In [ ]:
# from tqdm import tqdm
# for i in tqdm(X_train):
#   im = load_image2(i)
#   proprocess(im)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import os
import PIL
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.python.keras.layers import Dense, Flatten, Dropout, BatchNormalization
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam, SGD
from keras import callbacks

In [ ]:
# pretrained_model = tf.keras.applications.InceptionV3(
#     include_top=False,
#     # weights="imagenet",
#     input_shape=(224, 224, 3),
#     pooling='max',
#     classes= 2
# )

# for layer in pretrained_model.layers:
#   layer.trainable = False

In [ ]:
# model = Sequential()
# model.add(pretrained_model)
# model.add(Dense(1, activation = 'sigmoid'))

In [ ]:
from keras.models import Sequential
from keras.models import Model
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, EarlyStopping, ReduceLROnPlateau, TensorBoard
from keras import optimizers, losses, activations, models
from keras.layers import Convolution2D, Dense, Input, Flatten, Dropout, MaxPooling2D, BatchNormalization, GlobalAveragePooling2D, Concatenate
from keras import applications
input_shape = (224, 224, 3)
nclass = 2

base_model = tf.keras.applications.InceptionV3(weights='imagenet', 
                                include_top=False, 
                                input_shape=input_shape,
                                classes = 2)
base_model.trainable = False

add_model = Sequential()
add_model.add(base_model)
add_model.add(GlobalAveragePooling2D())
add_model.add(Dropout(0.5))
add_model.add(Dense(1, activation='sigmoid'))

model = add_model
model.compile(loss="binary_crossentropy",
              # optimizer=keras.optimizers.SGD(lr=1e-4, momentum=0.9),
              optimizer=Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-07, amsgrad=False, name="Adam"),
              metrics=['accuracy'])
model.summary()

87924736/87910968 [==============================] - 1s 0us/step
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 inception_v3 (Functional)   (None, 5, 5, 2048)        21802784  
                                                                 
 global_average_pooling2d (G  (None, 2048)             0         
 lobalAveragePooling2D)                                          
                                                                 
 dropout (Dropout)           (None, 2048)              0         
                                                                 
 dense (Dense)               (None, 1)                 2049      
                                                                 
Total params: 21,804,833
Trainable params: 2,049
Non-trainable params: 21,802,784
_________________________________________________________________


In [ ]:
# model.compile(optimizer=Adam(learning_rate=0.001,
#     beta_1=0.9,
#     beta_2=0.999,
#     epsilon=1e-07,
#     amsgrad=False,
#     name="Adam"), loss = 'binary_crossentropy', metrics = ['accuracy'])

In [ ]:
path = '/content/drive/MyDrive/'
early_stopping=callbacks.EarlyStopping(monitor='val_loss', min_delta=0, patience=10, verbose=0, mode='min')

# path_s = "/content/drive/MyDrive/30 อบรม superAI2/31 แข่งขัน/01 font recognition/train_model/"
filepath= path + "top_model_icv3.h5"

checkpoint = callbacks.ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=True, mode='min')

callbacks_list = [checkpoint]

In [ ]:
history = model.fit(X_train, y_train, 
                    validation_data= (X_test, y_test), 
                    batch_size=32, 
                    callbacks = callbacks_list,
                    epochs=50
                    )

Epoch 1/50
199/200 [============================>.] - ETA: 0s - loss: 0.7109 - accuracy: 0.5914
Epoch 00001: val_loss improved from inf to 0.56162, saving model to /content/drive/MyDrive/top_model_icv3.h5
200/200 [==============================] - 28s 80ms/step - loss: 0.7108 - accuracy: 0.5911 - val_loss: 0.5616 - val_accuracy: 0.7069
Epoch 2/50
140/200 [====================>.........] - ETA: 2s - loss: 0.6098 - accuracy: 0.6712

KeyboardInterrupt: ignored